In [8]:
import torch
import torch.nn as nn
import random

In [138]:
# change the list to a dictionary so that we can use integeres to represent the cards 
deck = [
    'Ace of Spades', '2 of Spades', '3 of Spades',
    '4 of Spades', '5 of Spades', '6 of Spades',
    '7 of Spades', '8 of Spades', '9 of Spades',
    '10 of Spades', 'Jack of Spades', 'Queen of Spades',
    'King of Spades',
    'Ace of Hearts', '2 of Hearts', '3 of Hearts',
    '4 of Hearts', '5 of Hearts', '6 of Hearts',
    '7 of Hearts', '8 of Hearts', '9 of Hearts',
    '10 of Hearts', 'Jack of Hearts', 'Queen of Hearts',
    'King of Hearts',
    'Ace of Diamonds', '2 of Diamonds', '3 of Diamonds',
    '4 of Diamonds', '5 of Diamonds', '6 of Diamonds',
    '7 of Diamonds', '8 of Diamonds', '9 of Diamonds',
    '10 of Diamonds', 'Jack of Diamonds', 'Queen of Diamonds',
    'King of Diamonds',
    'Ace of Clubs', '2 of Clubs', '3 of Clubs',
    '4 of Clubs', '5 of Clubs', '6 of Clubs',
    '7 of Clubs', '8 of Clubs', '9 of Clubs',
    '10 of Clubs', 'Jack of Clubs', 'Queen of Clubs',
    'King of Clubs'
  ]

card_dict = {}
for i, card in enumerate(deck): 
    card_dict[i+1] = card 
    
# randomly assign hands to each of the players in a n size game (max 9, min 4 players)

players = {}
used_keys = [] # avoids dealing the same card twice 

for i in range(1, 7): 
    player_cards = []
    for j in range(2): 
        key = random.choice(list(card_dict.keys())) # selects a random key 
        while key in used_keys: 
            key = random.choice(list(card_dict.keys())) # selects a new key if necessary
        value = card_dict.pop(key) # remove key-value pair from the dictionary
        used_keys.append(key) # add the key to the list of used keys 
        player_cards.append(value) 
    players[i] = player_cards 
    
    # after we run this we have to re-initialize the deck
    
print(players)

{1: ['9 of Clubs', '4 of Clubs'], 2: ['3 of Spades', '8 of Hearts'], 3: ['4 of Diamonds', '7 of Diamonds'], 4: ['Jack of Diamonds', '6 of Hearts'], 5: ['Ace of Spades', '3 of Clubs'], 6: ['3 of Hearts', 'Queen of Hearts']}


In [124]:
# now that each player can be dealt a hand, we need to set up the order of betting/blinds

big_blind_index = 1 
small_blind_index = 0

num_players = len(players)

def moveBlinds(): 
    for i in range(num_players): 
        print(f'Big blind is moving to: {players[big_blind_index + 1]}')
        print(f'Small blind is moving to: {players[small_blind_index + 1]}')
        
        big_blind_index += 1 
        small_blind_index += 1 
            
        # Reset the index if necessary
              
        if big_blind_index >= num_players: 
              big_blind_index = 0 
        if small_blind_index >= num_players: 
              small_blind_index = 0  
    

In [125]:
# define each starting stack

player_buyin = 20.00
big_blind_cost = 0.50 
small_blind_cost = 0.25

preflop = False # Preflop is between the dealing of the cards and the initiation of a flop 

pot = 0 

stacks = []

for player in players: 
    stacks.append(player_buyin)

print(stacks)


[20.0, 20.0, 20.0, 20.0, 20.0, 20.0]


In [126]:
# weight the hands so that the ai can play them 

def weight_cards(cards): 
    # dictionary to map string representation of ecah card in numerical value 
    values = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'Jack': 10, 'Queen': 10, 'King': 10, 'Ace': 11}
    
    # use list comprehension to iterate through the input string and sum the values of the cards 
    
    weights = sum([values[card.strip().split(' ')[0]] for card in cards])
    
    # TODO: if they are the same suit, add a small multiplier
    
    return weights 

for player in players: 
    print(weight_cards(players[player]))
    print(players[player])


    

11
['5 of Clubs', '6 of Clubs']
7
['4 of Hearts', '3 of Diamonds']
21
['Jack of Diamonds', 'Ace of Clubs']
20
['10 of Spades', 'Queen of Clubs']
13
['5 of Hearts', '8 of Spades']
17
['7 of Clubs', 'Jack of Spades']


In [127]:
def decisionAction(playerIndex): 
    # standard bet preflop is 1.5x the big blind 
    if preflop == True and pot == big_blind_cost + small_blind_cost: # if no one has bet yet
        betSize = 1.5*big_blind_cost
        stacks[playerIndex] -= betSize
        pot += betSize
    if preflop == True and pot != big_blind_cost + small_blind_cost: 
        # if someone else has already bet, we need to call, fold, raise
        if weight_cards(playerIndex) <= 15: 
            fold(playerIndex)
        if weight_cards(playerIndex) > 15: 
            call(playerIndex)

        

In [137]:
# play a pretend hand

preflop = True 
raisedPot = False 

def dealHand(): 
    
    num_players = len(players) 
    
    actionIndex = big_blind_index + 1 
    stacks[big_blind_index] -= big_blind_cost
    stacks[small_blind_index] -= small_blind_cost 
    pot = big_blind_cost + small_blind_cost
    
    while actionIndex < big_blind_index - num_players: 
        hand_weight = weight_cards(players[actionIndex])
        if hand_weight >= 18:
            bet_size = 1.5 * big_blind_cost
            stacks[actionIndex] -= bet_size
            pot += bet_size
            raisedPot = True
        
        
        else: 
            stacks[actionIndex] -= big_blind_cost
            pot += big_blind_cost
            
        '''       
        elif hand_weight > 11 and hand_weight < 18 and not raisedPot:
            stacks[actionIndex] -= big_blind_cost
            pot += big_blind_cost
        '''
  
        actionIndex += 1
            
    flop = []
    
    for j in range(3): 
        key = random.choice(list(card_dict.keys())) # selects a random key 
        while key in used_keys: 
            key = random.choice(list(card_dict.keys())) # selects a new key if necessary
        value = card_dict.pop(key) # remove key-value pair from the dictionary
        used_keys.append(key) # add the key to the list of used keys 
        flop.append(value) 
    
    return stacks, flop 

x = dealHand()
print(x)

([17.5, 15.0, 19.5, 19.25, 19.25, 19.5], ['4 of Clubs', 'King of Clubs', '3 of Spades'])


In [139]:
# reweight the hand after the flop comes down 

def reweight_hands(newCards, curCards): 
    pass




In [ ]:
class PokerNN(nn.Module): 
    def __init__(self): 
        super().__init__()
        self.layer1 = nn.Linear(in_features=7, out_features=64) 
        self.layer2 = nn.Linear(in_features=64, out_features=128)
        self.layer3 = nn.Linear(in_features=128, out_features=3) 
        
    def forward(self, x): 
        x = self.layer1(x) 
        x = torch.relu(x) 
        x = self.layer2(x) 
        x = torch.relu(x) 
        x = self.layer3(x) 
        x = torch.softmax(x, dim=1) 
        return x 
    
loss_fn = nn.CrossEntropyLoss() 

for inputs, labels in poker_dataset:
    # Pass the inputs through the neural network
    outputs = poker_nn(inputs)
    # Compute the loss
    loss = loss_fn(outputs, labels)
    # Update the weights and biases
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()